In [1]:
# MNIST Handwritten Digit Classification
import os
import torch
import torch.nn as nn               # to load neural network functionality
import torch.nn.functional as nnF       # Adds efficiency
from torch.utils.data import DataLoader # Data loading in batches
from torchvision import datasets, transforms
import sklearn
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix # For evaluating results
import matplotlib.pyplot as plt
%matplotlib inline
import time
torch.manual_seed(2002) # setting seed for consistent results


In [14]:
transformer = transforms.ToTensor()
#print(os.getcwd()) # For debugging :D lol folder problems
train_data = datasets.MNIST(root='\data', train=True, download=False, transform=transformer) # loading the training data
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: \data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [16]:
test_data = datasets.MNIST(root='\data', train=False, download=False, transform=transformer) # Loading the test data
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: \data
    Split: Test
    StandardTransform
Transform: ToTensor()